In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic
import re

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_csv("data/ARCOM.csv")

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_43113/1274466890.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/ARCOM.csv")


In [24]:
df

,N° Fiche,year,week,N° Version,Annonceur,Marque,Produit,Date,Date VP,Média,...,Hashtag,clean_script,mots_clefs_themes,mots_clefs_themes_str,visuel_str,visuel_clean,visuel_clean_bigram,clean_bigram_total,date_iso,month_nb
0,4704773,2014,1,332634386.0,NESTLE,GUIGOZ,GUIGOZ LAIT INFANTILE,01/01/2014,24/02/2012,TÉLÉVISION,...,NaN,"['faut', 'comprendre', 'bébés', 'comprendre', ...","CANARD , BAIN , INCOMPREHENSION , SE MOQUER,ON...",canard bain incomprehension se_moquer onomatop...,NaN,papa joue bébé bain essayer communiquer,papa joue bebe bain essayer communiquer,canard bain incomprehension se_moquer onomatop...,2014-01-01,1
1,4704855,2014,1,332634436.0,HENKEL,LE CHAT,LE CHAT EXPERT SAVON NOIR FLEUR ORANGE LIQUIDE,01/01/2014,05/08/2013,TÉLÉVISION,...,NaN,"['bords', 'méditerranée', 'secret', 'linge', '...","FLEUR D'ORANGER , SAVON NOIR , LESSIVE LIQUIDE...",fleur_d'oranger savon_noir lessive_liquide eff...,NaN,femme utiliser chat expert liquide savon noir ...,femme utiliser chat expert liquide savon noir ...,fleur_d'oranger savon_noir lessive_liquide eff...,2014-01-01,1
2,4704876,2014,1,332633131.0,RENAULT AUTOMOBILES,RENAULT CAPTUR,CAPTURCALL FR,01/01/2014,NaN,TÉLÉVISION,...,NaN,"['wou', 'monsieur', '15ème', 'arrange', 'masse...","CAMERA CACHEE,INTERROGATION, HUMOUR AUTRE, NOC...",camera_cachee interrogation humour_autre noctu...,NaN,caméra cacher homme invite personne prendre pl...,camera cacher homme invite personne prendre pl...,camera_cachee interrogation humour_autre noctu...,2014-01-01,1
3,4704886,2014,1,332634495.0,RECKITT BENCKISER,CALGON,CALGON 2EN1 GAMME,01/01/2014,NaN,TÉLÉVISION,...,NaN,"['drôle', 'odeur', 'traine', 'machine', 'compr...","DEPOT,LEADER, COMPARAISON, ODORAT, RECOMMANDE,...",depot leader comparaison odorat recommande tes...,NaN,technicien expliquer provenir mauvais odeur la...,technicien expliquer provenir mauvais_odeur la...,depot leader comparaison odorat recommande tes...,2014-01-01,1
4,4704934,2014,1,332634461.0,UPFIELD FRANCE,FRUIT D OR OMEGA 3,FRUIT D OR OMEGA 3&6,01/01/2014,19/11/2012,TÉLÉVISION,...,NaN,"['hugo', 'expert', 'graines', 'lin', 'tourneso...","OISEAU , MESANGE , COUPER LA PAROLE , SACHETS ...",oiseau mesange couper_la_parole sachets_de_gra...,NaN,homme appel hugo mésange aider fruit animal co...,homme appel hugo mesange aider fruit animal co...,oiseau mesange couper_la_parole sachets_de_gra...,2014-01-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50668,13862653,2022,9,364692645.0,RENAULT AUTOMOBILES,RENAULT ARKANA,RENAULT PRET A PARTIR E TECH HYBRIDE ARKANA HY...,28/02/2022,01/01/2022,TÉLÉVISION,...,NaN,"['papa', 'compliqué', 'expliquer', 'bosses', '...","REUNIONS,ENFANT, INNOVATION, LIEN PATERNEL, UN...",reunions enfant innovation lien_paternel unive...,NaN,homme travailler arrache pied renault bien rép...,homme travailler arrache pied renault bien rep...,reunions enfant innovation lien_paternel unive...,2022-02-28,98
50669,13862677,2022,9,364373993.0,MC DONALDS,MC DONALD S,MC DO GAMME ADULTES PROMO,28/02/2022,15/05/2017,TÉLÉVISION,...,NaN,"['moment', 'mc', 'donals', 'venez', 'découvrir...","ÉGO , FACILITÉ , DURÉE LIMITÉE , GÉNÉREUX , SO...",égo facilité durée_limitée généreux souvenirs ...,NaN,match football amateur homme avancer but vide ...,match_football amateur homme avancer_but vide_...,égo facilité durée_limitée généreux souvenirs ...,2022-02-28,98
50670,13862697,2022,9,364423316.0,SMART CUBE INVESTISSEMENT,SMART CUBE INVESTISSEMENT,SMART CUBE INVESTISSEMENT BATIMENTS INDUSTRIELS,28/02/2022,NaN,TÉLÉVISION,...,NaN,"['êtes', 'chef', 'entreprise', 'investisseur',...","CASQUE VR , ENTREPOT,EFFICACITE, CHOIX, INNOVA...",casque_vr entrepot efficacite choix innovation...,NaN,séquence montrer femme configurer bâtiment pro...,sequence_montrer femme configurer batiment pro...,casque_vr entrepot efficacite choix innovation...,2022-02-28,98
50671,13862734,2022,9,363844810.0,RECKITT BENCKISER HEALTHCARE,NUROFEN,NUROFEN FLASH,28/02/2022,07/01/2020,

In [5]:
df.columns

Index(['N° Fiche', 'year', 'week', 'N° Version', 'Annonceur', 'Marque',
       'Produit', 'Date', 'Date VP', 'Média', 'Medium', 'Catégorie', 'Support',
       'Format', 'Secteur', 'Classe', 'Groupe', 'Variété', 'N° Groupe/Variété',
       'Signature', 'Script', 'Incrustation', 'Titre', 'Libellé Version',
       'Visuel', 'Mots Clés', 'Thème', 'Musique / Auteur compositeur',
       'Musique / Titre', 'Agence', 'Réalisateur', 'Producteur', 'Récompense',
       'Musique / Version', 'Type de campagne', 'Musique / Interprète',
       'Partenaires', 'Titre émission', 'Hashtag', 'clean_script',
       'mots_clefs_themes', 'mots_clefs_themes_str', 'visuel_str',
       'visuel_clean', 'visuel_clean_bigram', 'clean_bigram_total', 'date_iso',
       'month_nb'],
      dtype='object')

In [30]:
for i, row in df[0:10].iterrows():
    print('-----------------------------')
    print(row['Annonceur'])
    print(row['Produit'])
    print('-----------------------------')
    print(row['Script'])

-----------------------------
NESTLE
GUIGOZ LAIT INFANTILE
-----------------------------
Voix femme : "Il faut comprendre les bébés pour comprendre leurs besoins. 
 ... Laboratoire GUIGOZ, parlons bébé."
-----------------------------
HENKEL
LE CHAT EXPERT SAVON NOIR FLEUR ORANGE LIQUIDE
-----------------------------
Voix homme : " Sur les bords de la Méditerranée, ...
 ... le secret d'un linge impeccable, c'est le savon noir ...
 ... reconnu pour ses vertus désincrustantes.
 ... En l'associant au doux parfum de la fleur d'oranger, ...
 ... LE CHAT créé pour vous la lessive LE CHAT L'EXPERT ...
 ... qui élimine les taches incrustées, ...
 ... tout en apportant une touche de fraîcheur à ...
 ... votre linge.
 ...LE CHAT L'EXPERT SAVON NOIR & FLEUR D'ORANGER. 
 
 ... Découvrez aussi LE CHAT SAVON DE MARSEILLE ET LAIT D'AMANDE DOUCE...
 ... Et LE CHAT BICARBONATE."
-----------------------------
RENAULT AUTOMOBILES
CAPTURCALL FR
-----------------------------
L'homme (1) : " Wou !
 (1) ... O

In [38]:
script = df.loc[1,'Script']

pattern = r'(\b(?:L\'homme|La femme)\s\(\d+\)\s?:[^"]+)'
matches = re.findall(pattern, script)

for match in matches: 
    print(match.strip())

In [39]:
script

'Voix homme : " Sur les bords de la Méditerranée, ...\n ... le secret d\'un linge impeccable, c\'est le savon noir ...\n ... reconnu pour ses vertus désincrustantes.\n ... En l\'associant au doux parfum de la fleur d\'oranger, ...\n ... LE CHAT créé pour vous la lessive LE CHAT L\'EXPERT ...\n ... qui élimine les taches incrustées, ...\n ... tout en apportant une touche de fraîcheur à ...\n ... votre linge.\n ...LE CHAT L\'EXPERT SAVON NOIR & FLEUR D\'ORANGER. \n \n ... Découvrez aussi LE CHAT SAVON DE MARSEILLE ET LAIT D\'AMANDE DOUCE...\n ... Et LE CHAT BICARBONATE."'